## Object-Frame Mapping Driver Function

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import wordnet
import json

"""
Takes in JSON files for image objects and for frames outputted from sling. Organizes according to image ID.
"""
def mapping_fxn_test(img_json, frames_json):
    res = {}
    with open(frames_json) as file1:
        with open(img_json) as file2:
            frames = json.load(file1)
            objs = json.load(file2)
            
            filtered_objs = {}
            for img_id in frames:
                tmp = {}
                filtered_objs[str(img_id)] = []
                file_name = 'test_images/'+img_id+'.jpg'
                for obj in objs[file_name]:
                    obj_name = obj['name']
                    box_points = obj['box_points']
                    tmp[str(obj_name)] = box_points
                filtered_objs[str(img_id)] = tmp
            print(filtered_objs)
            print('\n')
            print(frames)
            
            # similarity
            mapping = {}
            for img in frames:
                most_similar_word = None
                max_similarity = 0
                org_obj = None
                tmp = {}
                for f in frames[img]: 
                    for obj in filtered_objs[img]:
                        _obj_name = obj.split()[0] if len(obj.split()) > 1 else obj
                        
                        s1 = wordnet.synsets(_obj_name)[0]
                        s2 = wordnet.synsets(f)[0]
                        s = wordnet.wup_similarity(s1,s2)
                        print('similarity between ' + obj + ' and ' + f + ': ' + str(s))
                        if max_similarity < s and s > 0.5:
                            print(f + ' and ' + _obj_name + ' has high similarity: ' + str(s))
                            max_similarity = s
                            most_similar_word = f
                            org_obj = obj
                    if most_similar_word is not None:
                        #tmp = {org_obj: str(most_similar_word)}
                        mapping[org_obj] = str(most_similar_word)
                        #mapping[img] = tmp
                print(mapping)
                tmp = {}
                for o in filtered_objs[img]:
                    if o not in mapping or mapping[o] not in frames[img]:
                        continue
                    print(img)
                    print('eek ' + mapping[o])
                    arg, v = frames[img][mapping[o]][0]
                    tmp[mapping[o]] = [str(arg), str(v), filtered_objs[img][o]]
                res[str(img)] = tmp
    return res

In [ ]:
mapping_fxn_test('test_resources/test_images_obj.json', 'test_resources/output.json')

{'2': {'person': [464, 169, 525, 327], 'sandwich': [148, 120, 249, 175], 'chair': [426, 264, 513, 327], 'cup': [366, 290, 411, 340]}, '3': {'tv': [244, 72, 392, 196], 'person': [0, 11, 267, 287], 'potted plant': [382, 122, 468, 248], 'book': [340, 247, 476, 293], 'cup': [224, 167, 249, 198], 'cell phone': [257, 213, 300, 237], 'mouse': [301, 226, 336, 243]}, '7': {'person': [258, 386, 2612, 3964], 'laptop': [987, 2371, 2424, 3240], 'chair': [65, 2312, 1322, 3974]}, '10': {'person': [210, 274, 1413, 1542], 'sports ball': [908, 1335, 1238, 1600]}, '12': {'person': [0, 111, 564, 436], 'wine glass': [295, 298, 598, 420], 'tie': [216, 397, 280, 432]}}


{'2': {'food': [['arg1', 'eat-01']], 'person': [['arg0', 'eat-01']]}, '3': {'boy': [['arg1', 'sit-01']], 'chair': [['arg2', 'sit-01']], 'television': [['argm-loc', 'sit-01']]}, '7': {'text': [['arg1', 'close-01']], 'woman': [['arg0', 'close-01']]}, '10': {'boy': [['arg0', 'kick-01']], 'soccer': [['arg1', 'kick-01']]}, '12': {'man': [['arg0',

{'10': {'boy': ['arg0', 'kick-01', [210, 274, 1413, 1542]],
  'soccer': ['arg1', 'kick-01', [908, 1335, 1238, 1600]]},
 '12': {'man': ['arg0', 'hold-01', [0, 111, 564, 436]],
  'wine': ['arg1', 'hold-01', [295, 298, 598, 420]]},
 '2': {'food': ['arg1', 'eat-01', [148, 120, 249, 175]],
  'person': ['arg0', 'eat-01', [464, 169, 525, 327]]},
 '3': {'boy': ['arg1', 'sit-01', [0, 11, 267, 287]],
  'television': ['argm-loc', 'sit-01', [244, 72, 392, 196]]},
 '7': {'woman': ['arg0', 'close-01', [258, 386, 2612, 3964]]}}

In [2]:
import json
word_to_pos_dict = {'10': {'boy': ['arg0', 'kick-01', [210, 274, 1413, 1542]],
  'soccer': ['arg1', 'kick-01', [908, 1335, 1238, 1600]]},
 '12': {'wine': ['arg1', 'hold-01', [295, 298, 598, 420]]},
 '2': {'food': ['arg1', 'eat-01', [148, 120, 249, 175]],
  'person': ['arg0', 'eat-01', [464, 169, 525, 327]]},
 '3': {'boy': ['arg1', 'sit-01', [0, 11, 267, 287]],
  'television': ['argm-loc', 'sit-01', [244, 72, 392, 196]]},
 '7': {'woman': ['arg0', 'close-01', [258, 386, 2612, 3964]]}}

with open("word_to_pos.json", "w") as outfile:
    json.dump(word_to_pos_dict, outfile)